<a href="https://colab.research.google.com/github/peterm-m/TFM/blob/master/Selvar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sys

#X[TIEMPO,VARIABLE]



def selvar_python(X,maxlag,bs,itmax):
  (t_max,d)=X.shape
  lags=np.zeros((d,d))
  scores=np.zeros((d,d))
  lags_final=np.zeros((d,d))
  for i in range(d):
    print(i)
    prss=getscore(X,lags,i,maxlag,bs,t_max,d,'PRSS')
    print(prss)
    iteracion=0
    condition=True
    while condition:
      iteracion+=1
      (sel,lagsel,nuevo_prss)=mejor_actualizacion(X,i,t_max,d,prss,lags,maxlag,bs)
      if sel>0:
        prss=nuevo_prss
        lags[i,sel]=lagsel
        sel==0
      if iteracion>=itmax:
        condition=False
      if sel==0:
        condition=False
    scores[i,:]=puntuaciones(X,lags,i,maxlag,bs,t_max,d)
    
  return scores,lags_final



def mejor_actualizacion(X,x_i,t_max,d,prss,lags,maxlag,bs):
  '''
  Input:
    X datos
    x_i variable a estimar
    t_max tiempo maximo
    d numero de variables
    press puntuacion mejor modelo
    lags lags del modelo 
  Output:
    sel: variable seleccionada
    l:lag seleccionado
    nuevo_prss: mejor puntuacion
  '''
  (sel,lagsel,prss)=(0,0,prss)
  nuevo_prss=0
  for x_j in range(d):
    for l in range(1,maxlag):
      if x_j==x_i or lags[x_i,x_j]==l:
        continue
      tmp=lags[x_i,x_j]
      lags[x_i,x_j]=l
      nuevo_prss=getscore(X,lags,x_i,maxlag,bs,t_max,d,'PRSS')  
      if nuevo_prss<prss:
        (sel,lagsel,prss)=(x_j,l,nuevo_prss )
      lags[x_i,x_j]=tmp
  return sel,lagsel,nuevo_prss


def getscore(X , lags, x_i, maxlag, bs,t_max,d, tipo, variable='nada'):
  '''
  Input:
datos = valores de las series temporales
lags  = lags del modelo para la variable x_i
x_i   = variable a predecir
maxlag= valor maximo del lag
tipo= RSS  o  PRSS
variable= si es necesario quitar alguna variable del modelo.
  Output:
prss= predicted RSS  para la variable.
  '''
  if (maxlag>t_max or maxlag<1): maxlag=1
  if (bs<0): bs=int((t_max-maxlag)/(-bs))
  if (bs>(t_max-maxlag)): bs=t_max-maxlag


  score_modelo=0
  epochs=int((t_max-maxlag)/bs)
  lags=lags[x_i,:]
  if variable!='nada':
    lags[variable]=0

  for epoch in range(epochs):

    inicio=maxlag+epoch*bs
    final =maxlag+(epoch+1)*bs

    X_i=X[inicio :final,x_i]
    mask=lags>0

    X_modelo=X[:,mask]
    lags_tanda=lags[mask]
    X_js=np.zeros((bs,X_modelo.shape[1]))

    for x_j in range(X_js.shape[1]):

      X_js[:,x_j]=X[int(inicio-lags_tanda[x_j]):int(final-lags_tanda[x_j]),x_j]
    score_modelo = score_modelo+score(X_js,X_i,tipo)
    
  return score_modelo




def score(X,Y,tipo):
  '''
  Input:
  X atributos
  Y clases
  tipo RSS o PRSS
  '''
  hat=X.dot(np.linalg.pinv(X))
  res=Y-hat.dot(Y)
  if tipo=='PRSS':
    sqr=np.square(res/(1-np.diagonal(hat)))
    return sqr.sum()
  elif tipo=='RSS':
    return np.square(res).sum()
  else:
    print('error')


def puntuaciones(X,lags,i,maxlag,bs,t_max,d):
  
  scores=np.zeros((1,d))
  score=getscore(X,lags,i,maxlag,bs,t_max,d,'RSS')
  for j in range(d):
    if j==i:
      continue
    if lags[i,j]>0:
      scores[0,j]=getscore(X,lags,i,maxlag,bs,t_max,d,'RSS',variable=j)-score
  return scores
  